In [ ]:
import sys
sys.path.append('../../')
sys.path.append('../common')

In [ ]:
from controller import Controller
from control.pure_pursuit import PurePursuit
from control.PID import PID 
from vehicle import Vehicle
from track import Track
from simulation import Simulation, show_img

vehicle = Vehicle(x=0, y=0, velocity=0, heading=0, wheel_base=2.6)
track = Track()
controller = Controller()

In [3]:
from ipywidgets import RadioButtons
posctl = RadioButtons(options=['PID', 'PurePursuit'],
                   value = 'PID',   
                   description='', disabled=False)
velctl = RadioButtons(options=['なし（速度一定）', 'PID'],
                   value = 'PID',   
                   description='', disabled=False)

print("軌道追従アルゴリズムを選択")
display(posctl)
print("速度制御アルゴリズムを選択")
display(velctl)

軌道追従アルゴリズムを選択


RadioButtons(options=('PID', 'PurePursuit'), value='PID')

速度制御アルゴリズムを選択


RadioButtons(index=1, options=('なし（速度一定）', 'PID'), value='PID')

In [4]:
from ipywidgets import FloatSlider

Kp, Ki, Kd, Kdd = None, None, None, None

def ask_PID_parameters():
        Kp = FloatSlider(value=3, min=0, max=10.0, step=0.1, description='Kp=', readout_format='.1f',)
        Ki = FloatSlider(value=0, min=0, max=10.0, step=0.1, description='Ki=', readout_format='.1f',)
        Kd = FloatSlider(value=0, min=0, max=10.0, step=0.1, description='Kd=', readout_format='.1f',)
        display(Kp)
        display(Ki)
        display(Kd)

def ask_PurePursuit_parameters():
        Kdd = FloatSlider(value=3, min=0, max=10.0, step=0.1, description='Kdd=', readout_format='.1f',)
        display(Kdd)

In [5]:
if posctl.value == 'PID':
        print("軌道追従アルゴリズム(PID)のパラメータを設定")
        ask_PID_parameters()
        controller.set_handle_controller(PID(Kp=Kp.value, Ki=Ki.value, Kd=Kd.value, set_point=0))
elif posctl.value == 'PurePursuit':
        print("軌道追従アルゴリズム(PurePursuit)のパラメータを設定")
        ask_PurePursuit_parameters()
        controller.set_handle_controller(PurePursuit(Kdd=Kdd.value))

軌道追従アルゴリズム(PID)のパラメータを設定


FloatSlider(value=3.0, description='Kp=', max=10.0, readout_format='.1f')

FloatSlider(value=0.0, description='Ki=', max=10.0, readout_format='.1f')

FloatSlider(value=0.0, description='Kd=', max=10.0, readout_format='.1f')

In [7]:
if posctl.value == 'PID':
        print("速度制御アルゴリズム(速度一定)のパラメータを設定")
        desired_velocity = FloatSlider(value=25, min=0, max=30.0, step=0.1, description='目標速度=', readout_format='.1f',)
        display(desired_velocity)
elif posctl.value == 'PurePursuit':
        print("軌道追従アルゴリズム(PID)のパラメータを設定")
        desired_velocity = FloatSlider(value=25, min=0, max=30.0, step=0.1, description='目標速度=', readout_format='.1f',)
        display(desired_velocity)
        ask_PID_parameters()
        controller.set_handle_controller(PID(Kp=Kp.value, Ki=Ki.value, Kd=Kd.value, set_point=desired_velocity))

速度制御アルゴリズム(速度一定)のパラメータを設定


FloatSlider(value=25.0, description='目標速度=', max=30.0, readout_format='.1f')

In [ ]:
from IPython.display import clear_output

sim = Simulation(vehicle, Track(), controller)

img_list = []
for i in range(1,1000):
    try:
        sim.step()
        # visualization
        img = sim.cv_plot()
        if i%2==0:
          img_list.append(img)
          show_img(img)
        clear_output(wait=True)
        # check for simulation end
        if len(sim.waypoints) < 10:
            break

    except KeyboardInterrupt:
        break